In [1]:
import pandas as pd
import numpy as np

# Correct data from the provided table
data = {
    'Margin': [3, 7, 6, 10, 4, 14, 1, 2, 8, 5, 17, 21, 13, 11, 18, 20, 24, 16, 28, 9, 12, 15, 27, 19, 31, 23, 25, 22, 26, 35, 29, 34, 30, 32, 38, 37, 33, 0, 41, 40, 42, 45, 36, 39, 43, 49, 44, 46, 48, 50, 52, 55, 58, 59],
    'Occurrence': [937, 571, 374, 350, 310, 308, 253, 249, 235, 219, 205, 174, 173, 152, 144, 137, 136, 132, 105, 101, 101, 96, 81, 70, 70, 67, 66, 60, 50, 40, 35, 33, 31, 31, 28, 23, 17, 14, 11, 10, 7, 7, 5, 4, 4, 3, 2, 2, 2, 1, 1, 1, 1, 1],
    'Percentage': [15.02, 9.15, 5.99, 5.61, 4.97, 4.94, 4.05, 3.99, 3.77, 3.51, 3.29, 2.79, 2.77, 2.44, 2.31, 2.2, 2.18, 2.12, 1.68, 1.62, 1.62, 1.54, 1.3, 1.12, 1.12, 1.07, 1.06, 0.96, 0.8, 0.64, 0.56, 0.53, 0.5, 0.5, 0.45, 0.37, 0.27, 0.22, 0.18, 0.16, 0.11, 0.11, 0.08, 0.06, 0.06, 0.05, 0.03, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02]
}

df = pd.DataFrame(data)
df = df.sort_values('Margin').reset_index(drop=True)

# Calculate total games
total_games = df['Occurrence'].sum()

# Function to calculate winning percentage for whole points
def calc_whole_point(margin, df, total_games):
    games_under = df.loc[df['Margin'] < margin, 'Occurrence'].sum()
    games_over = total_games - games_under - df.loc[df['Margin'] == margin, 'Occurrence'].sum()
    return ((games_under / 2) / (games_under + games_over) + 0.5) * 100

# Function to calculate winning percentage for half points
def calc_half_point(margin, df, total_games):
    games_under_and_at = df.loc[df['Margin'] <= margin, 'Occurrence'].sum()
    return ((games_under_and_at / 2) / total_games + 0.5) * 100

# Calculate winning percentages for whole and half points
results = []
for margin in np.arange(0, 59.5, 0.5):
    if margin.is_integer():
        win_pct = calc_whole_point(margin, df, total_games)
        occurrence = df.loc[df['Margin'] == margin, 'Occurrence'].values[0] if margin in df['Margin'].values else '-'
        percentage = df.loc[df['Margin'] == margin, 'Percentage'].values[0] if margin in df['Margin'].values else '-'
        games_under = df.loc[df['Margin'] < margin, 'Occurrence'].sum()
        games_over = total_games - games_under - df.loc[df['Margin'] == margin, 'Occurrence'].sum()
        calculation = f"((({games_under} ) / ({games_under} + {games_over}))/2) + 0.5 * 100"
    else:
        win_pct = calc_half_point(margin - 0.5, df, total_games)
        occurrence = '-'
        percentage = '-'
        games_under_and_at = df.loc[df['Margin'] <= margin - 0.5, 'Occurrence'].sum()
        calculation = f"((({games_under_and_at} ) / {total_games})/2) + 0.5 * 100"
    
    results.append({
        'Margin': margin,
        'Occurrence': occurrence,
        'Percentage': percentage,
        'Winning_Percentage': win_pct,
        'Calculation': calculation
    })

results_df = pd.DataFrame(results)

# Calculate deltas
results_df['Delta'] = results_df['Winning_Percentage'].diff()

# Format percentages and deltas
for col in ['Percentage', 'Winning_Percentage']:
    results_df[col] = results_df[col].apply(lambda x: f"{x:.2f}%" if x != '-' else x)
results_df['Delta'] = results_df['Delta'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "")

print(results_df.to_string(index=False))

 Margin Occurrence Percentage Winning_Percentage                               Calculation Delta
    0.0         14      0.22%             50.00%       (((0 ) / (0 + 6226))/2) + 0.5 * 100      
    0.5          -          -             50.11%            (((14 ) / 6240)/2) + 0.5 * 100 0.11%
    1.0        253      4.05%             50.12%     (((14 ) / (14 + 5973))/2) + 0.5 * 100 0.00%
    1.5          -          -             52.14%           (((267 ) / 6240)/2) + 0.5 * 100 2.02%
    2.0        249      3.99%             52.23%   (((267 ) / (267 + 5724))/2) + 0.5 * 100 0.09%
    2.5          -          -             54.13%           (((516 ) / 6240)/2) + 0.5 * 100 1.91%
    3.0        937     15.02%             54.87%   (((516 ) / (516 + 4787))/2) + 0.5 * 100 0.73%
    3.5          -          -             61.64%          (((1453 ) / 6240)/2) + 0.5 * 100 6.78%
    4.0        310      4.97%             62.25% (((1453 ) / (1453 + 4477))/2) + 0.5 * 100 0.61%
    4.5          -          - 